# The Capital Problem
Notebook to show the the consequences of not properly understanding the lifetime dynamics of the capital requirements of derivatives.

In [126]:
import numpy as np

class IR_Swap:
    def __init__(self, notional, start_year, end_year, fixed_rate, direction):
        self.notional = notional 
        self.start = start_year
        self.end = end_year
        self.fixed = fixed_rate
        self.direction = direction
    def pv(self, curve, date = 0):
        cf = self.cashflows(curve, date)
        pv = 0
        year = 1
        for flow in cf:
            pv += curve.df(year)*cf[year-1]
            year += 1
        pv = pv / curve.df(date)
        return pv
    def cashflows(self, curve, date = 0):
        cf = []
        if self.direction == "p":
            direction = -1
        elif self.direction == "r":
            direction = 1
        else:
            direction = None
        for year in range(1, self.end + 1):
            if year <= self.start or year <= date:
                cf.append(0)
            else:
                float = curve.forward(year)
                fixed = self.fixed
                notl = self.notional
                cf.append(direction * (fixed - float) * notl)
        return cf
            
        
class IR_Curve:
    def __init__(self, rates):
        self.rates = rates
    def curve_all(self):
        df = []
        lib = []
        df.append(1)
        lib.append(np.nan)
        for rate in self.rates:
            disc_fac = (1 - rate * (sum(df)-1)) / (1 + rate)
            df.append(disc_fac)
            libor = (df[-2] / df[-1]) - 1
            lib.append(libor)
        return [df, lib]
    def df(self, year):
        return self.curve_all()[0][year] 
    def forward(self, year):
        return self.curve_all()[1][year]
    
        


In [127]:
swap = IR_Swap(100000000, 0, 10, 0.028, "p")
curve = IR_Curve([0.01,0.012,0.014,0.016,0.018,0.020,0.022,0.024,0.026,0.028])
#print(curve.curve_all())
swap.cashflows(curve)
swap.pv(curve, 5)

5227264.524759646